In [23]:
import numpy as np
import pandas as pd
from datetime import datetime

In [16]:
def read_catalog(filename):
    with open(filename, 'r') as f:
        # read header
        header = f.readline().strip().split()
        # read data
        data = []
        for line in f:
            fields = line.strip().split('|')
            fields[1] = pd.to_datetime(fields[1])
            data.append(fields)
    # create pandas dataframe
    df = pd.DataFrame(data, columns=['EventID', 'Time', 'Latitude', 'Longitude', 'Depth/Km', 'Author', 'Catalog', 'Contributor', 'ContributorID', 'MagType', 'Magnitude', 'MagAuthor', 'EventLocationName', 'EventType'])
    # convert latitude, longitude, depth, and magnitude columns to numeric format
    df[['Latitude', 'Longitude', 'Depth/Km', 'Magnitude']] = df[['Latitude', 'Longitude', 'Depth/Km', 'Magnitude']].apply(pd.to_numeric)
    df = df.rename(columns={'EventID': 'ID'})
    df = df.rename(columns={'Latitude': 'Lat'})
    df = df.rename(columns={'Longitude': 'Long'})
    df = df.rename(columns={'Depth/Km': 'Depth'})
    df = df.rename(columns={'Magnitude': 'Mag'})
    return df

# example usage
file = 'norcia16.txt'
earthquake_catalog = read_catalog(file)
print(earthquake_catalog.head())

        ID                    Time      Lat     Long  Depth         Author  \
0  6898861 2016-07-03 04:00:45.170  42.5677  13.3212   13.7    SURVEY-INGV   
1  6899891 2016-07-03 11:26:13.290  42.9975  12.9073    9.1  BULLETIN-INGV   
2  6899921 2016-07-03 11:42:33.880  42.6328  13.0067    9.9    SURVEY-INGV   
3  6901011 2016-07-03 20:10:41.110  42.4693  13.2998   12.8  BULLETIN-INGV   
4  6901031 2016-07-03 21:10:35.470  42.4608  13.3023   13.4  BULLETIN-INGV   

  Catalog Contributor ContributorID MagType  Mag MagAuthor  \
0                                        ML  1.1        --   
1                                        ML  1.5        --   
2                                        ML  1.0        --   
3                                        ML  1.6        --   
4                                        ML  1.2        --   

                   EventLocationName   EventType  
0             4 km W Campotosto (AQ)  earthquake  
1          8 km W Monte Cavallo (MC)  earthquake  
2  5 

In [17]:
print(earthquake_catalog['Mag'])

0       1.1
1       1.5
2       1.0
3       1.6
4       1.2
       ... 
9995    1.4
9996    1.3
9997    0.9
9998    0.8
9999    1.0
Name: Mag, Length: 10000, dtype: float64


In [19]:
from tlp_fs import calc_Mc_b
import numpy as np

def calc_Mc_b(mag,plot=0):
    nNumberMagnitude=np.floor(mag.max()*10)+1
    xx=np.linspace(np.floor(mag.min()*10)/10,mag.max(),int((np.floor(mag.max()*10)/10-
                                                            np.floor(mag.min()*10)/10)*10)+1)
    vhist,vMagBins=np.histogram(mag,bins=xx)
    iMc = np.where(vhist==vhist.max())[0][-1]
    fMc=vMagBins[iMc]
    hist = vhist[iMc:][::-1]
    bins = vMagBins[iMc:][::-1]
    cum_hist = hist.cumsum()
    log_cum_sum = np.log10(cum_hist)
    bins = bins[1:]
    b,a = np.polyfit(bins, log_cum_sum, 1)
    if plot==1:
        plt.figure()
        plt.subplot(211)
        plt.bar(vMagBins[:-1]+0.05,vhist,0.1)
        plt.axvline(x=fMc,c='r',lw=2)
        plt.title('Mc:'+ "{:.2f}".format(fMc)+'  '+'b value:'+  "{:.2f}".format(-b))
        plt.xlim(-1,8)
        plt.ylabel('Event number')
        plt.subplot(212)
        plt.plot(bins,np.log10(cum_hist))
        plt.plot(bins,a + b*bins)
        plt.xlim(-1,8)
        plt.xlabel('Magnitude')
        plt.ylabel('log10(CDF)')
        plt.show()
        
    return fMc, a, -b

calc_Mc_b(earthquake_catalog['Mag'])

(1.2999999999999998, 5.1095882047730345, 0.9358256956052159)

In [ ]:
slide = []
bDates = []
ewLens = []

timeStart = 2016.5#np.min(time_year)
timeEnd = 2017#np.max(time_year) #or custom decimal year dates
timeTotal = timeEnd - timeStart

#areaParams = (dist_along_proj<70)&(dist_along_proj>0)&(dist_norm_proj<5)&(dist_norm_proj>-5)
#timeParams = (time_year>timeStart)&(time_year<timeEnd)

bigMag = 5

#data = np.where(areaParams&timeParams)
#data = np.where(timeParams&areaParams&(mag<=bigMag))
index = earthquake_catalog['Mag']
print(len(index),index)

ewSize = 100 #number of eqs per window

for i in range(len(index)):

    ew = index[i:i+ewSize] # filter 'ew' by 'wParams'
    ewEnd = time_year[ew][-1]
    bDates.append(time_year[ew[-1]]) # append last event date in 'ew'. Provided by ChatGPT

    bEW = calc_Mc_b(mag[ew])[2]
    addbVal_slide(slide,bEW,bDates)

    ewLens.append((np.max(time_year[ew])-np.min(time_year[ew])))

    if ew[-1] >= max(index):
        print(f"end of loop, steps: {i}; date of last eq: {ew[-1]}; last b: {bEW}")
        xMin = np.min(bDates)
        xMax = np.max(bDates)
        print(f"x range: {xMin} to {xMax}; array lengths: {len(slide),len(bDates)}")
        plt.plot(bDates,ewLens,"ro",markersize=0.5,label="Window size (in years)")
        addbVal_slide(slide,bEW,bDates,xMin,xMax,1)
        break

bigEqs = np.where(areaParams&timeParams&(mag>bigMag))
bigDates = time_year[bigEqs]

In [36]:
d = earthquake_catalog['Time'][1]
d_fixed = d.to_pydatetime()

print(f"did thi work{d.to_pydatetime()} ORRRR {d_fixed}")

#time_str = '2016-07-03 04:00:45.170'
time_str = d_fixed

dt = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S.%f')
year_start = datetime(dt.year, 1, 1)
delta = dt - year_start
decimal_year = dt.year + delta.days / 365.25 + delta.seconds / (365.25 * 24 * 3600)

print(decimal_year)


did thi work2016-07-03 11:26:13.290000 ORRRR 2016-07-03 11:26:13.290000


TypeError: strptime() argument 1 must be str, not datetime.datetime